# Load dependencies

In [ ]:
!pip install pycaret
!pip install sweetviz

In [ ]:
import pandas as pd
import sweetviz as sv

from pycaret.regression import *

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv').drop("id", axis=1)#.sample(frac=0.05)
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv').drop("id", axis=1)
sample_submission = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

# Exploratory Data Analysis

In [ ]:
my_report = sv.analyze(train, target_feat='loss')

In [ ]:
my_report.show_notebook()

# Model Training

In [ ]:
exp_clf = setup(train, 
                target="loss",
                silent= True)

In [ ]:
N_SELECT = 5
K=5
SCORE_METRIC='RMSE'

In [ ]:
# return best model
best = compare_models(sort = SCORE_METRIC, n_select=N_SELECT,
                      exclude = ['et', 'rf', 'svm'], fold=K)

# Model Ensembling

In [ ]:
blended = blend_models(estimator_list = best, fold=K, optimize=SCORE_METRIC)

In [ ]:
final_model = finalize_model(blended)

# Model Analysis

In [ ]:
plot_model(final_model)

# Sub

In [ ]:
# Final prediction
pred_esb = predict_model(final_model, test)

# Get predictions
sample_submission['loss'] = pred_esb['Label']

# Write submission
sample_submission.to_csv('submission.csv',index=False)